In [1]:
import tensorflow as tf

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
#define weight,bias
def weight_variable(shape):
    init = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)
def bias_variable(shape):
    init = tf.constant(0.1,shape=shape)
    return tf.Variable(init)

In [2]:
#placeholder
xs = tf.placeholder(tf.float32,[None,9000])
ys = tf.placeholder(tf.float32,[None,1])
keep_prob = tf.placeholder(tf.float32)
xs_image = tf.reshape(xs,[-1,30,300,1])

In [3]:
#define cnn layer
def cov2d(W,x):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')

In [4]:
#define max-pooling
def  max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
#conv1 layer
#每一个层都有自己单独的weight和bias 
w_conv1 = weight_variable([5,5,1,32])
bias_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(cov2d(w_conv1,xs_image)+bias_conv1)
p_conv1 = max_pool_2x2(h_conv1)

In [8]:
#conv2
w_conv2 = weight_variable([5,5,32,64])
bias_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(cov2d(w_conv2,p_conv1)+bias_conv2)
p_conv2 = max_pool_2x2(h_conv2)

In [9]:
#func1
w_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])

#flat
h_pool_flat = tf.reshape(p_conv2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,w_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)




In [10]:
#func2
w_fc2 = weight_variable([1024,1])
b_fc2 = bias_variable([1])


prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

In [11]:
#define accuracy

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pred = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1})
    correct = tf.equal(tf.argmax(y_pred,1),tf.argmax(v_ys,1))
    correct_accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    result = sess.run(correct_accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

In [12]:
#cost
cost = tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=ys)


#train
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)





Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
init = tf.global_variables_initializer()

In [14]:
import pandas as pd

In [15]:
test = pd.read_csv('./data/bert_test.txt',delimiter='\t',names=["label","review"])
val = pd.read_csv('./data/bert_val.txt',delimiter='\t',names=["label","review"])
train = pd.read_csv('./data/bert_train.txt',delimiter='\t',names=["label","review"])



In [16]:
train.shape,val.shape,train.shape

((18000, 2), (9000, 2), (18000, 2))

In [17]:
#load data
x_test = test.review
y_test = test.label
x_train = train.review
y_train = train.label

In [18]:
import jieba

In [19]:
#load dictionary
def loadWord2Vec(filename):
    vocab = []
    embd = []
    cnt = 0
    fr = open(filename,'r')
    line = fr.readline().strip()
    #print line
    word_dim = int(line.split(' ')[1])    
    vocab.append("unk")
    embd.append([0]*word_dim)
    for line in fr :
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print("loaded word2vec") 
    fr.close()
    return vocab,embd



In [20]:
import numpy as np
vocab,embd = loadWord2Vec('/Users/james/Downloads/sgns.zhihu.word')
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

loaded word2vec


In [21]:
def dividesentence(a):
    sentence = []
    seg_list = jieba.cut(a, cut_all=True)
    for item in seg_list:
        sentence.append(item)
    return sentence

In [22]:
def findemdding(str1):
    for i in range(0,vocab_size):
        if vocab[i] == str1:
            return embedding[i]
    else:
        return np.zeros(shape=(300,))

In [23]:
def combinevector(b):
    sentenceembed = []
    if len(b)<=30:
        for i in range(0,len(b)):
            c = findemdding(b[i])
            sentenceembed.append(c)
        return i,np.array(sentenceembed)
    else:
        for i in range(0,30):
            c = findemdding(b[i])
            sentenceembed.append(c)
        return i,np.array(sentenceembed)

In [24]:
def getvect(a):
    b = dividesentence(a)
    count, arr = combinevector(b)
    if count<=30:
        a =  np.zeros(shape=(30-count-1,300))
        d = np.insert(arr ,count+1,values=a,axis=0)
        return d
    else:
        return arr

In [25]:
x_test_vec = []
for i in range(0,3000):
    a = getvect(x_test[i])
    print(i)
    b = a.reshape([1,9000])
    print(b)
    x_test_vec.append(b)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vr/3b_f9c5j0yvgq7_41pp6_ktm0000gn/T/jieba.cache
Loading model cost 18.245 seconds.
Prefix dict has been built succesfully.


0
[['-0.328553' '0.275700' '-0.028519' ... '0.0' '0.0' '0.0']]
1
[['0.029680' '0.306241' '0.282320' ... '0.0' '0.0' '0.0']]
2
[['-0.017388' '0.584633' '0.103035' ... '0.0' '0.0' '0.0']]
3
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
4
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
5
[['-0.175772' '0.400553' '-0.254192' ... '0.0' '0.0' '0.0']]
6
[['-0.133331' '0.597004' '0.176164' ... '0.077148' '0.736683' '-0.061945']]
7
[['-0.122848' '0.497776' '-0.391459' ... '0.0' '0.0' '0.0']]
8
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
9
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
10
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
11
[['-0.328553' '0.275700' '-0.028519' ... '0.0' '0.0' '0.0']]
12
[['-0.122848' '0.497776' '-0.391459' ... '0.0' '0.0' '0.0']]
13
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
14
[['-0.529631' '0.156119' '-0.485743' ... '0.0' '0.0' '0.0']]
15
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
16
[['0.053887' '0.234212' 

134
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
135
[['-0.106512' '0.224250' '-0.127244' ... '0.0' '0.0' '0.0']]
136
[['-0.146676' '0.258798' '-0.160531' ... '0.0' '0.0' '0.0']]
137
[['0.071185' '0.101118' '-0.251992' ... '0.0' '0.0' '0.0']]
138
[['-0.529631' '0.156119' '-0.485743' ... '0.0' '0.0' '0.0']]
139
[['0.053887' '0.234212' '-0.025752' ... '0.0' '0.0' '0.0']]
140
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
141
[['0.117474' '0.351831' '-0.264038' ... '0.0' '0.0' '0.0']]
142
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
143
[['-0.236725' '0.201288' '-0.091106' ... '0.0' '0.0' '0.0']]
144
[['-0.146676' '0.258798' '-0.160531' ... '0.0' '0.0' '0.0']]
145
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
146
[['-0.455289' '0.519651' '-0.212719' ... '0.0' '0.0' '0.0']]
147
[['-0.332395' '-0.115595' '0.409784' ... '0.0' '0.0' '0.0']]
148
[['-0.146676' '0.258798' '-0.160531' ... '0.0' '0.0' '0.0']]
149
[['-0.209840' '0.154647' '-0.207592' ... '

268
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
269
[['-0.080844' '0.365711' '0.026342' ... '0.0' '0.0' '0.0']]
270
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
271
[[0. 0. 0. ... 0. 0. 0.]]
272
[['-0.368740' '0.214759' '0.190465' ... '0.0' '0.0' '0.0']]
273
[['-0.455289' '0.519651' '-0.212719' ... '0.0' '0.0' '0.0']]
274
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
275
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
276
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
277
[['0.226384' '0.407340' '-0.274968' ... '0.0' '0.0' '0.0']]
278
[['-0.326380' '0.550656' '-0.403863' ... '0.0' '0.0' '0.0']]
279
[['-0.100047' '0.514658' '-0.443213' ... '0.0' '0.0' '0.0']]
280
[['0.109008' '0.222542' '-0.618695' ... '0.0' '0.0' '0.0']]
281
[['-0.248224' '0.094396' '-0.181514' ... '0.0' '0.0' '0.0']]
282
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
283
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
284
[['-0.169243' '0.028708' '-0.159108' ... '0.0' '0.0' '0.0']]
285
[

400
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
401
[['0.259146' '-0.094650' '-0.023178' ... '0.0' '0.0' '0.0']]
402
[['-0.073035' '0.511388' '-0.202659' ... '0.0' '0.0' '0.0']]
403
[['-0.136755' '0.435059' '-0.212545' ... '0.0' '0.0' '0.0']]
404
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
405
[['-0.326380' '0.550656' '-0.403863' ... '0.0' '0.0' '0.0']]
406
[['-0.070275' '0.227458' '-0.207198' ... '0.0' '0.0' '0.0']]
407
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
408
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
409
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
410
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
411
[['-0.130600' '-0.327653' '-0.153970' ... '0.0' '0.0' '0.0']]
412
[['-0.080083' '0.577106' '-0.257534' ... '0.0' '0.0' '0.0']]
413
[['0.057250' '0.394978' '-0.368708' ... '0.0' '0.0' '0.0']]
414
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
415
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
416
[['0.158812' '0.489716' '0.101925' ... '0.

...

2413
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2414
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
2415
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2416
[['0.403931' '0.112383' '0.003231' ... '0.0' '0.0' '0.0']]
2417
[['-0.160741' '0.458292' '-0.170375' ... '0.0' '0.0' '0.0']]
2418
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
2419
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2420
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
2421
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
2422
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
2423
[['-0.475714' '0.110802' '-0.065556' ... '0.0' '0.0' '0.0']]
2424
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
2425
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
2426
[['-0.293463' '0.269916' '0.027578' ... '0.0' '0.0' '0.0']]
2427
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
2428
[['-0.138979' '0.087155' '0.070907' ... '0.0' '0.0' '0.0']]
24

2553
[['-0.140343' '-0.086717' '0.016051' ... '0.0' '0.0' '0.0']]
2554
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2555
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
2556
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
2557
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
2558
[['-0.407651' '0.268911' '-0.092309' ... '0.0' '0.0' '0.0']]
2559
[['0.248365' '-0.022684' '0.171322' ... '0.0' '0.0' '0.0']]
2560
[['-0.441043' '0.140380' '-0.064710' ... '0.0' '0.0' '0.0']]
2561
[['0.158812' '0.489716' '0.101925' ... '0.0' '0.0' '0.0']]
2562
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2563
[['-0.062814' '0.620987' '-0.007332' ... '0.0' '0.0' '0.0']]
2564
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
2565
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
2566
[['-0.072055' '0.296364' '-0.302396' ... '0.0' '0.0' '0.0']]
2567
[['-0.053734' '0.312588' '-0.418957' ... '0.0' '0.0' '0.0']]
2568
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]


2690
[['-0.146731' '0.233013' '-0.089547' ... '0.0' '0.0' '0.0']]
2691
[['0.176519' '-0.094249' '-0.448472' ... '0.0' '0.0' '0.0']]
2692
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
2693
[['-0.183694' '0.064124' '0.135140' ... '0.0' '0.0' '0.0']]
2694
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2695
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2696
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
2697
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
2698
[['-0.122848' '0.497776' '-0.391459' ... '0.0' '0.0' '0.0']]
2699
[['-0.182882' '0.179686' '-0.082053' ... '0.0' '0.0' '0.0']]
2700
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2701
[['-0.073035' '0.511388' '-0.202659' ... '0.0' '0.0' '0.0']]
2702
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
2703
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
2704
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2705
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2706
[['-0.403142' '0.295281' '

2832
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2833
[['0.226384' '0.407340' '-0.274968' ... '0.0' '0.0' '0.0']]
2834
[['-0.403142' '0.295281' '-0.464814' ... '0.0' '0.0' '0.0']]
2835
[['-0.271951' '-0.417269' '-0.187528' ... '0.0' '0.0' '0.0']]
2836
[['-0.073035' '0.511388' '-0.202659' ... '0.0' '0.0' '0.0']]
2837
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
2838
[['-0.055911' '-0.006339' '0.208331' ... '0.0' '0.0' '0.0']]
2839
[['0.158812' '0.489716' '0.101925' ... '0.0' '0.0' '0.0']]
2840
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
2841
[['-0.299959' '0.299669' '-0.144580' ... '0.0' '0.0' '0.0']]
2842
[['-0.028444' '0.152092' '-0.058855' ... '0.0' '0.0' '0.0']]
2843
[['-0.293463' '0.269916' '0.027578' ... '0.0' '0.0' '0.0']]
2844
[['-0.105065' '0.756901' '0.131612' ... '0.0' '0.0' '0.0']]
2845
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
2846
[['-0.069798' '0.008636' '-0.191884' ... '0.0' '0.0' '0.0']]
2847
[['-0.326003' '0.425066' '-

2968
[['-0.053806' '0.322857' '0.084941' ... '0.0' '0.0' '0.0']]
2969
[['0.150244' '0.352785' '-0.088605' ... '0.0' '0.0' '0.0']]
2970
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
2971
[['-0.299236' '0.356827' '-0.093667' ... '0.0' '0.0' '0.0']]
2972
[['-0.069798' '0.008636' '-0.191884' ... '0.0' '0.0' '0.0']]
2973
[['-0.070275' '0.227458' '-0.207198' ... '0.0' '0.0' '0.0']]
2974
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
2975
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
2976
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
2977
[['0.226384' '0.407340' '-0.274968' ... '0.0' '0.0' '0.0']]
2978
[['0.150244' '0.352785' '-0.088605' ... '0.0' '0.0' '0.0']]
2979
[['-0.355702' '0.258416' '0.020941' ... '0.0' '0.0' '0.0']]
2980
[['-0.307803' '0.175746' '-0.297423' ... '0.0' '0.0' '0.0']]
2981
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
2982
[['-0.122848' '0.497776' '-0.391459' ... '0.0' '0.0' '0.0']]
2983
[['0.262178' '0.066907' '0.02

In [26]:
y_train = y_train[:8000]

In [27]:
x_train_vec = []
for i in range(0,8000):
    a = getvect(x_train[i])
    print(i)
    b = a.reshape([1,9000])
    print(b)
    x_train_vec.append(b)

0
[['-0.441043' '0.140380' '-0.064710' ... '0.0' '0.0' '0.0']]
1
[['-0.078367' '-0.124955' '0.013711' ... '0.0' '0.0' '0.0']]
2
[['-0.129149' '0.364760' '-0.053860' ... '0.0' '0.0' '0.0']]
3
[['-0.268900' '0.283240' '-0.206830' ... '0.0' '0.0' '0.0']]
4
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
5
[['0.117474' '0.351831' '-0.264038' ... '0.0' '0.0' '0.0']]
6
[['0.005664' '0.407404' '-0.153532' ... '0.0' '0.0' '0.0']]
7
[['0.150244' '0.352785' '-0.088605' ... '0.0' '0.0' '0.0']]
8
[['-0.332395' '-0.115595' '0.409784' ... '0.0' '0.0' '0.0']]
9
[['-0.216616' '0.325066' '-0.131592' ... '0.0' '0.0' '0.0']]
10
[['-0.075654' '0.515432' '0.187096' ... '0.0' '0.0' '0.0']]
11
[['-0.234850' '0.356982' '-0.333283' ... '0.0' '0.0' '0.0']]
12
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
13
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
14
[['-0.529631' '0.156119' '-0.485743' ... '0.0' '0.0' '0.0']]
15
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
16
[['0.

139
[['0.196227' '0.099232' '0.377239' ... '0.0' '0.0' '0.0']]
140
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
141
[['-0.209840' '0.154647' '-0.207592' ... '0.0' '0.0' '0.0']]
142
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
143
[['0.286699' '0.774587' '-0.038948' ... '0.0' '0.0' '0.0']]
144
[['-0.007712' '0.509682' '-0.037688' ... '0.0' '0.0' '0.0']]
145
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
146
[['-0.053734' '0.312588' '-0.418957' ... '0.0' '0.0' '0.0']]
147
[['-0.355702' '0.258416' '0.020941' ... '0.0' '0.0' '0.0']]
148
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
149
[['-0.175772' '0.400553' '-0.254192' ... '0.0' '0.0' '0.0']]
150
[['-0.154072' '0.157986' '0.099474' ... '0.0' '0.0' '0.0']]
151
[['-0.182882' '0.179686' '-0.082053' ... '0.0' '0.0' '0.0']]
152
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
153
[['-0.046944' '0.087344' '0.011195' ... '0.0' '0.0' '0.0']]
154
[['-0.368740' '0.214759' '0.190465' ... '0.0' '0.0' '0.0']]
155
[['-0.228192' '0.474353' '0.2373

278
[['-0.112581' '0.458949' '0.227390' ... '0.0' '0.0' '0.0']]
279
[['0.0' '0.0' '0.0' ... '0.067685' '-0.027377' '-0.470059']]
280
[['-0.183694' '0.064124' '0.135140' ... '0.0' '0.0' '0.0']]
281
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
282
[['0.204110' '0.346302' '-0.120510' ... '0.0' '0.0' '0.0']]
283
[['0.005664' '0.407404' '-0.153532' ... '0.0' '0.0' '0.0']]
284
[['0.242701' '0.253519' '-0.467724' ... '0.0' '0.0' '0.0']]
285
[['-0.441043' '0.140380' '-0.064710' ... '0.0' '0.0' '0.0']]
286
[['-0.070275' '0.227458' '-0.207198' ... '0.0' '0.0' '0.0']]
287
[['0.005664' '0.407404' '-0.153532' ... '0.0' '0.0' '0.0']]
288
[['-0.127875' '-0.101700' '-0.272671' ... '0.0' '0.0' '0.0']]
289
[['-0.112581' '0.458949' '0.227390' ... '0.0' '0.0' '0.0']]
290
[['-0.112581' '0.458949' '0.227390' ... '0.0' '0.0' '0.0']]
291
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
292
[['0.226384' '0.407340' '-0.274968' ... '0.0' '0.0' '0.0']]
293
[['-0.273875' '0.577425' '-0.493764' ... '0.0'

414
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
415
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
416
[['-0.291533' '0.565492' '0.050881' ... '0.0' '0.0' '0.0']]
417
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
418
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
419
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
420
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
421
[['-0.036373' '-0.089171' '0.142755' ... '0.0' '0.0' '0.0']]
422
[['0.035009' '0.296007' '-0.335060' ... '0.0' '0.0' '0.0']]
423
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
424
[['-0.133331' '0.597004' '0.176164' ... '0.0' '0.0' '0.0']]
425
[['-0.604690' '0.071966' '-0.251474' ... '0.0' '0.0' '0.0']]
426
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
427
[['-0.452859' '0.574260' '0.093137' ... '0.0' '0.0' '0.0']]
428
[['-0.122848' '0.497776' '-0.391459' ... '0.0' '0.0' '0.0']]
429
[['0.158812' '0.489716' '0.101925' ... '0.0' '0.0' '0.0']]
430
[['0.226384' '0.

549
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
550
[['0.057962' '0.136641' '-0.353504' ... '0.0' '0.0' '0.0']]
551
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
552
[['-0.293463' '0.269916' '0.027578' ... '0.0' '0.0' '0.0']]
553
[['0.005664' '0.407404' '-0.153532' ... '0.0' '0.0' '0.0']]
554
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
555
[['-0.080083' '0.577106' '-0.257534' ... '0.0' '0.0' '0.0']]
556
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
557
[['-0.154072' '0.157986' '0.099474' ... '0.0' '0.0' '0.0']]
558
[['0.029680' '0.306241' '0.282320' ... '0.0' '0.0' '0.0']]
559
[['0.053243' '0.613648' '-0.313202' ... '0.0' '0.0' '0.0']]
560
[['-0.744735' '0.682391' '-0.160387' ... '0.0' '0.0' '0.0']]
561
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
562
[['-0.368740' '0.214759' '0.190465' ... '0.0' '0.0' '0.0']]
563
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
564
[['0.053243' '0.613648' '-0.313202' ... '0.0' '0.0' '0.0']]
565
[['-0.122848' '0.497776' '-0.3914

...

7439
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
7440
[['-0.138979' '0.087155' '0.070907' ... '0.0' '0.0' '0.0']]
7441
[['-0.312306' '0.109963' '0.164818' ... '0.0' '0.0' '0.0']]
7442
[['-0.141891' '0.119332' '-0.350027' ... '0.0' '0.0' '0.0']]
7443
[['0.008060' '0.191082' '-0.192758' ... '0.0' '0.0' '0.0']]
7444
[['-1.008323' '1.155053' '-0.633397' ... '0.0' '0.0' '0.0']]
7445
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
7446
[['-0.407651' '0.268911' '-0.092309' ... '0.0' '0.0' '0.0']]
7447
[['0.226384' '0.407340' '-0.274968' ... '0.0' '0.0' '0.0']]
7448
[['-0.441043' '0.140380' '-0.064710' ... '0.0' '0.0' '0.0']]
7449
[[0. 0. 0. ... 0. 0. 0.]]
7450
[['0.673984' '0.642777' '-0.258970' ... '0.0' '0.0' '0.0']]
7451
[['0.226384' '0.407340' '-0.274968' ... '0.210005' '0.172321' '-0.113025']]
7452
[['-0.183694' '0.064124' '0.135140' ... '0.0' '0.0' '0.0']]
7453
[['-0.069798' '0.008636' '-0.191884' ... '0.0' '0.0' '0.0']]
7454
[['-0.234850' '0.356982' '-0.333

7574
[['0.158812' '0.489716' '0.101925' ... '0.0' '0.0' '0.0']]
7575
[['-0.309170' '0.032893' '-0.565046' ... '0.0' '0.0' '0.0']]
7576
[['-0.324496' '0.408859' '-0.121158' ... '0.0' '0.0' '0.0']]
7577
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7578
[['-0.368740' '0.214759' '0.190465' ... '0.0' '0.0' '0.0']]
7579
[['0.222968' '0.203990' '-0.121025' ... '0.0' '0.0' '0.0']]
7580
[['-0.441043' '0.140380' '-0.064710' ... '0.0' '0.0' '0.0']]
7581
[['-0.228192' '0.474353' '0.237316' ... '0.0' '0.0' '0.0']]
7582
[['-0.154072' '0.157986' '0.099474' ... '0.0' '0.0' '0.0']]
7583
[['-0.326003' '0.425066' '-0.205147' ... '0.0' '0.0' '0.0']]
7584
[['-0.069798' '0.008636' '-0.191884' ... '0.0' '0.0' '0.0']]
7585
[['0.053243' '0.613648' '-0.313202' ... '0.0' '0.0' '0.0']]
7586
[['0.053243' '0.613648' '-0.313202' ... '0.0' '0.0' '0.0']]
7587
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
7588
[['0.117474' '0.351831' '-0.264038' ... '0.0' '0.0' '0.0']]
7589
[['0.206187' '0.645184' '-0.342

7713
[['0.005664' '0.407404' '-0.153532' ... '0.0' '0.0' '0.0']]
7714
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7715
[['-0.116923' '-0.189205' '-0.445969' ... '0.0' '0.0' '0.0']]
7716
[['-0.162159' '0.277196' '-0.081780' ... '0.0' '0.0' '0.0']]
7717
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7718
[['-0.218195' '0.112689' '-0.038075' ... '0.0' '0.0' '0.0']]
7719
[['-0.073035' '0.511388' '-0.202659' ... '0.0' '0.0' '0.0']]
7720
[['-0.097701' '0.289202' '-0.030196' ... '0.0' '0.0' '0.0']]
7721
[['0.038902' '0.130619' '-0.218247' ... '0.0' '0.0' '0.0']]
7722
[['0.110767' '0.231101' '-0.314310' ... '0.0' '0.0' '0.0']]
7723
[['0.150244' '0.352785' '-0.088605' ... '0.0' '0.0' '0.0']]
7724
[['-0.128812' '0.567472' '0.329079' ... '0.0' '0.0' '0.0']]
7725
[['0.228256' '0.290297' '0.040570' ... '0.0' '0.0' '0.0']]
7726
[['0.316118' '0.114735' '-0.209474' ... '0.0' '0.0' '0.0']]
7727
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
7728
[['-0.542511' '0.450695' '-0.178781' ... '0.0'

7857
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7858
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7859
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7860
[['-0.326380' '0.550656' '-0.403863' ... '0.0' '0.0' '0.0']]
7861
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7862
[['-0.097701' '0.289202' '-0.030196' ... '0.424660' '-0.012973'
  '-0.115294']]
7863
[['-0.229122' '0.538295' '-0.065930' ... '0.0' '0.0' '0.0']]
7864
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7865
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
7866
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7867
[['-0.299236' '0.356827' '-0.093667' ... '0.0' '0.0' '0.0']]
7868
[['-0.130383' '0.063972' '0.076819' ... '0.0' '0.0' '0.0']]
7869
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7870
[['-0.368740' '0.214759' '0.190465' ... '0.0' '0.0' '0.0']]
7871
[['-0.292765' '0.410053' '0.107178' ... '0.0' '0.0' '0.0']]
7872
[['-0.052082' '0.280218' '-0.361067' ... '0.0' '0.0' '0.0']]
7873
[['-0.293463' '0.269916' '0.027578' ... '0

7993
[['0.150244' '0.352785' '-0.088605' ... '0.0' '0.0' '0.0']]
7994
[['-0.166823' '0.440631' '-0.202386' ... '0.0' '0.0' '0.0']]
7995
[['0.158812' '0.489716' '0.101925' ... '0.0' '0.0' '0.0']]
7996
[['-0.309170' '0.032893' '-0.565046' ... '0.0' '0.0' '0.0']]
7997
[['-0.309170' '0.032893' '-0.565046' ... '0.0' '0.0' '0.0']]
7998
[['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']]
7999
[['0.215692' '0.318854' '-0.222328' ... '0.0' '0.0' '0.0']]


In [28]:
x_train_vec_array =  np.array(x_train_vec)

In [29]:
x_train_vec_array_reshape=x_train_vec_array.reshape([8000,9000])

In [30]:
y_train_vec = []
for i in range(0,8000):
    a=np.array(y_train[i]).reshape([1,1])
    y_train_vec.append(a)

In [31]:
y_train_vec_array = np.array(y_train_vec)

In [32]:
y_train_vec_array_reshape = y_train_vec_array.reshape([8000,1])

In [ ]:
#sess.run
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        
        sess.run(train_op,feed_dict={xs:x_train_vec_array_reshape,ys:y_train_vec_array_reshape,keep_prob:0.5})
        #if i % 50 == 0:
            #print(compute_accuracy(np.array(x_test_vec),y_test))